In [4]:
# Import packages
import pandas as pd
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())
import matplotlib.pyplot as plt
import json

In [5]:
# Load the business.json file
with open("C:\\Users\\Raymond Loh\\OneDrive\\Documents\\yelp_academic_dataset_business.json", encoding="utf-8") as dataset_business:
    data = [json.loads(line) for line in dataset_business]
# Create DataFrame from the loaded data
df_b = pd.DataFrame(data)

In [6]:
# Filter the DataFrame to include only businesses that are still open
df_b = df_b[df_b['is_open'] == 1]
# Split the categories and explode them into separate rows, then filter for coffee-related categories
df_explode = df_b.assign(categories=df_b.categories.str.split(', ')).explode('categories')
coffee_categories = df_explode[df_explode.categories.str.contains('Coffee', case=True, na=False)]
# Count the occurrences of coffee-related categories
coffee_category_counts = coffee_categories.categories.value_counts()
# Further filter the original DataFrame for specific coffee-related business categories
business_coffee = df_b[df_b['categories'].str.contains(
    'Coffee & Tea|Coffee Roasteries|Coffeeshops|Coffee & Tea Supplies',
    case=False, na=False)]

In [7]:
# Define the path to the Yelp review dataset
review_json_path = 'C:\\Users\\Raymond Loh\\OneDrive\\Documents\\yelp_academic_dataset_review.json'
# Define the chunk size for processing the data
size = 1000000
# Initialize a list to store processed chunks
chunk_list = []
# Load the dataset in chunks and process each chunk
for chunk in pd.read_json(review_json_path, lines=True, 
                          dtype={'review_id': str, 'user_id': str, 'business_id': str, 
                                 'stars': int, 'date': str, 'text': str, 
                                 'useful': int, 'funny': int, 'cool': int}, 
                          chunksize=size):
    # Drop unnecessary columns from the chunk
    chunk = chunk.drop(['review_id', 'useful', 'funny', 'cool'], axis=1)
    # Rename the 'stars' column to 'review_stars' to avoid conflicts
    chunk = chunk.rename(columns={'stars': 'review_stars'})
    # Merge the chunk with the business dataset to keep only relevant reviews
    chunk_list.append(pd.merge(business_coffee, chunk, on='business_id', how='inner'))
# Concatenate all processed chunks into a single DataFrame
df = pd.concat(chunk_list, ignore_index=True)
# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

In [10]:
import pandas as pd
from pandasql import sqldf

# Initialize pysqldf
pysqldf = lambda q: sqldf(q, globals())


def descriptive_stats(table_name, table_col):
    print("==============================")
    print(f"Table Name: {table_name}")
    print(f"Table Col: {table_col}")
    print("==============================")
    
    print("")
    
    # Count
    cnt = pysqldf(f"""SELECT "COUNT" as Stat, count({table_col}) as Value 
                FROM {table_name};""")
    
    # Mean
    mean = pysqldf(f"""SELECT "MEAN" as Stat, 
                CAST(AVG({table_col}) as int) as Value 
                FROM {table_name}""")
    
    # Median
    median = pysqldf(f"""SELECT "MEDIAN" as Stat, {table_col} as Value 
                FROM {table_name}
                ORDER BY {table_col}
                LIMIT 1
                OFFSET (SELECT COUNT(*) FROM {table_name}) / 2 """)
    
    # Mode
    mode = pysqldf(f"""SELECT "MODE" as Stat, 
                {table_col} as Value,
                COUNT(*) as Count
                FROM {table_name}
                GROUP BY {table_col}
                ORDER BY COUNT(*) DESC
                LIMIT 1
                """)
    
    # Min and Max
    min_value = pysqldf(f"""SELECT 'MIN' as Stat, MIN({table_col}) as Value FROM {table_name}""")
    max_value = pysqldf(f"""SELECT 'MAX' as Stat, MAX({table_col}) as Value FROM {table_name}""")
    
    # Concatenate and display results
    results = pd.concat([cnt, mean, median, min_value, max_value]).set_index("Stat")
    display(results)
    display(mode.set_index('Stat'))
    print("")

# Example usage with DataFrame 'df'
descriptive_stats('df', 'stars')
descriptive_stats('df', 'review_count')
descriptive_stats('df', 'review_stars')

Table Name: df
Table Col: stars



,Value
Stat,
COUNT,351634.0
MEAN,3.0
MEDIAN,4.0
MIN,1.0
MAX,5.0


,Value,Count
Stat,,
MODE,4.5,125866



Table Name: df
Table Col: review_count



,Value
Stat,
COUNT,351634
MEAN,453
MEDIAN,185
MIN,5
MAX,5721


,Value,Count
Stat,,
MODE,5721,5778



Table Name: df
Table Col: review_stars



,Value
Stat,
COUNT,351634
MEAN,3
MEDIAN,5
MIN,1
MAX,5


,Value,Count
Stat,,
MODE,5,176118


In [11]:
import pandas as pd
from pandasql import sqldf

# Initialize pysqldf
pysqldf = lambda q: sqldf(q, globals())

def descriptive_stats(table_name, table_col):
    print("="*30)
    print(f"Table Name: {table_name}")
    print(f"Column: {table_col}")
    print("="*30)
    
    # SQL queries for descriptive statistics
    queries = {
        "COUNT": f"""SELECT 'COUNT' as Stat, COUNT({table_col}) as Value FROM {table_name}""",
        "MEAN": f"""SELECT 'MEAN' as Stat, ROUND(AVG({table_col}), 2) as Value FROM {table_name}""",
        "MEDIAN": f"""SELECT 'MEDIAN' as Stat, {table_col} as Value 
                      FROM {table_name}
                      ORDER BY {table_col}
                      LIMIT 1 OFFSET (SELECT COUNT(*) FROM {table_name}) / 2""",
        "MODE": f"""SELECT 'MODE' as Stat, {table_col} as Value 
                    FROM {table_name}
                    GROUP BY {table_col}
                    ORDER BY COUNT(*) DESC
                    LIMIT 1""",
        "MIN": f"""SELECT 'MIN' as Stat, MIN({table_col}) as Value FROM {table_name}""",
        "MAX": f"""SELECT 'MAX' as Stat, MAX({table_col}) as Value FROM {table_name}""",
    }
    
    # Execute queries and collect results
    results = pd.concat([pysqldf(query) for query in queries.values()]).set_index('Stat')
    
    # Display results with improved styling
    display(results.style.set_table_styles([
        {'selector': 'th', 'props': [('font-weight', 'bold'), ('background-color', '#f7f7f9')]},
        {'selector': 'td', 'props': [('padding', '8px'), ('border', '1px solid #ddd')]}
    ]))
    
    print("")  # Blank line for separation

# Example usage with DataFrame 'df'
descriptive_stats('df', 'stars')
descriptive_stats('df', 'review_count')
descriptive_stats('df', 'review_stars')

Table Name: df
Column: stars


,Value
Stat,
COUNT,351634.000000
MEAN,3.900000
MEDIAN,4.000000
MODE,4.500000
MIN,1.000000
MAX,5.000000



Table Name: df
Column: review_count


,Value
Stat,
COUNT,351634.000000
MEAN,453.220000
MEDIAN,185.000000
MODE,5721.000000
MIN,5.000000
MAX,5721.000000



Table Name: df
Column: review_stars


,Value
Stat,
COUNT,351634.000000
MEAN,3.900000
MEDIAN,5.000000
MODE,5.000000
MIN,1.000000
MAX,5.000000


In [44]:
# List the coffee company popular operating hour
pd.set_option('display.max_colwidth', None)
pysqldf("""SELECT hours, COUNT(*) as total_review
        FROM df
        GROUP BY hours
        ORDER BY total_review DESC
        LIMIT 5
        """)

,hours,total_review
0,"{""Monday"": ""0:0-0:0"", ""Tuesday"": ""0:0-0:0"", ""Wednesday"": ""0:0-0:0"", ""Thursday"": ""0:0-0:0"", ""Friday"": ""0:0-0:0"", ""Saturday"": ""0:0-0:0"", ""Sunday"": ""0:0-0:0""}",18048
1,"{""Monday"": ""8:0-18:0"", ""Tuesday"": ""8:0-18:0"", ""Wednesday"": ""8:0-18:0"", ""Thursday"": ""8:0-18:0"", ""Friday"": ""8:0-18:0"", ""Saturday"": ""8:0-18:0"", ""Sunday"": ""8:0-18:0""}",6647
2,"{""Monday"": ""7:0-14:0"", ""Tuesday"": ""7:0-14:0"", ""Wednesday"": ""7:0-14:0"", ""Thursday"": ""7:0-14:0"", ""Friday"": ""7:0-14:0"", ""Saturday"": ""7:0-14:0"", ""Sunday"": ""7:0-14:0""}",4038
3,"{""Monday"": ""7:0-15:0"", ""Tuesday"": ""7:0-15:0"", ""Wednesday"": ""7:0-15:0"", ""Thursday"": ""7:0-15:0"", ""Friday"": ""7:0-15:0"", ""Saturday"": ""7:0-15:0"", ""Sunday"": ""7:0-15:0""}",3676
4,"{""Monday"": ""8:0-14:0"", ""Tuesday"": ""8:0-14:0"", ""Wednesday"": ""8:0-14:0"", ""Thursday"": ""8:0-14:0"", ""Friday"": ""8:0-14:0"", ""Saturday"": ""8:0-14:0"", ""Sunday"": ""8:0-14:0""}",3225


In [55]:
# List the cities with average review rating
pysqldf("""SELECT city, AVG(review_stars) as avg_rating, COUNT(*) as total_review
        FROM df
        GROUP BY city
        ORDER BY total_review DESC, avg_rating
        """)

,city,avg_rating,total_review
0,Philadelphia,4.050741,63006
1,New Orleans,4.075420,36091
2,Nashville,3.923305,24252
3,Tampa,3.910321,23584
4,Tucson,3.857586,18685
...,...,...,...
486,Chester,4.000000,5
487,Cumberland,4.000000,5
488,Eastampton Township,4.200000,5
489,Mehlville,4.400000,5


In [12]:
# List the cities with the most reviews
pysqldf("""SELECT city, COUNT(*) as total_review
        FROM df
        GROUP BY state
        ORDER BY total_review DESC
        """)

,city,total_review
0,Philadelphia,85155
1,Tampa,56429
2,New Orleans,41923
3,Nashville,31388
4,St. Louis,24925
5,Fishers,23054
6,Santa Barbara,20251
7,Sparks,19637
8,Tucson,19514
9,Pennsville,9441


In [50]:
# List the cities with max review rating
pysqldf("""
    SELECT city, MAX(review_stars) as max_rating, COUNT(*) as total_review
    FROM df
    GROUP BY city
    ORDER BY max_rating DESC, max_rating
""")

,city,max_rating,total_review
0,Abington,5,204
1,Affton,5,76
2,Alton,5,192
3,Ambler,5,342
4,Antioch,5,445
...,...,...,...
486,Cahokia,3,8
487,Stanton,3,8
488,Silverdale,2,11
489,Westwego,2,9


In [51]:
# List the cities with min review rating
pysqldf("""
    SELECT city, MIN(review_stars) as min_rating, COUNT(*) as total_review
    FROM df
    GROUP BY city
    ORDER BY min_rating DESC, min_rating
""")

,city,min_rating,total_review
0,Bradenton,5,17
1,Deptford Township,5,6
2,Kimmswick,5,5
3,Terrytown,5,6
4,Danville,4,10
...,...,...,...
486,Yeadon,1,5
487,Zephyrhills,1,206
488,Zieglerville,1,13
489,Zionsville,1,261


In [56]:
pd.reset_option('display.max_colwidth')
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,user_id,review_stars,text,date
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{""RestaurantsDelivery"": ""False"", ""OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{""Monday"": ""7:0-20:0"", ""Tuesday"": ""7:0-20:0"", ...",6_SpY41LIHZuIaiDs5FMKA,4,This is nice little Chinese bakery in the hear...,2014-05-26 01:09:53
1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{""RestaurantsDelivery"": ""False"", ""OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{""Monday"": ""7:0-20:0"", ""Tuesday"": ""7:0-20:0"", ...",tCXElwhzekJEH6QJe3xs7Q,4,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06
2,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{""RestaurantsDelivery"": ""False"", ""OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{""Monday"": ""7:0-20:0"", ""Tuesday"": ""7:0-20:0"", ...",WqfKtI-aGMmvbA9pPUxNQQ,5,"A delightful find in Chinatown! Very clean, an...",2013-10-25 01:34:57
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{""RestaurantsDelivery"": ""False"", ""OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{""Monday"": ""7:0-20:0"", ""Tuesday"": ""7:0-20:0"", ...",3-1va0IQfK-9tUMzfHWfTA,5,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57
4,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{""RestaurantsDelivery"": ""False"", ""OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{""Monday"": ""7:0-20:0"", ""Tuesday"": ""7:0-20:0"", ...",EouCKoDfzaVG0klEgdDvCQ,4,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...,2013-10-25 02:31:35
